In [1]:
import spacy 
import re 
import pandas as pd
from spacy.lang.en import English
from collections import Counter
from itertools import chain
import ahocorasick
import ast
from analyze_m_data import get_occupations_list_en, get_occupations_list
from spacy.lang.de import German
from spacy.lang.it import Italian
from spacy.lang.es import Spanish 
import matplotlib.pyplot as plt
import itertools
from collections import Counter
from operator import itemgetter

In [24]:
de_female_df = pd.read_csv('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/sets_for_analysis/de_female_occ_all_sentences_fulldata.csv', encoding="utf-8")

In [4]:
print(de_female_df.columns)

Index(['Unnamed: 0', 'wikidata_code', 'title', 'intro', 'gender',
       'occupations', 'first_sentence', 'named_entities',
       'overlapping_occupations_male',
       'overlapping_occupations_sentence1_male',
       'overlapping_occupations_female',
       'overlapping_occupations_sentence1_female'],
      dtype='object')


In [26]:
def new_german_analysis(df):
    #Want to: 
    # 1. Split up english occupation titles list into two lists: one token occupations and multi token
    # 2. For one token occupations, want to find matches (in first sentence) with hard word boundaries (like in german code)
    # 3. For multi token occupations, want to just look up matches 
    # 4. Want to remove named entities from matched tokens in that line 
    # 5. Want to get final matched titles list 
    df['intro'] = df['intro'].astype(str)
    df['first_sentence'] = df['first_sentence'].astype(str)
    df['named_entities'] = df['named_entities'].apply(ast.literal_eval)
    df['named_entities'] = df['named_entities'].apply(lambda x: [item.lower() for item in x])
    df['overlapping_occupations_sentence1_female'] = df['overlapping_occupations_sentence1_female'].apply(ast.literal_eval)
    df['overlapping_occupations_sentence1_male'] = df['overlapping_occupations_sentence1_male'].apply(ast.literal_eval)
    print("hello")
    print(df['named_entities'].head(10))
    
    def filter_NER(row):
        col1_counts = Counter(row['overlapping_occupations_sentence1_male'])  # Count occurrences in col1
        col2_counts = Counter(row['named_entities'])  # Count occurrences in col2

        # Remove only up to the number of times they appear in col2
        for word, count in col2_counts.items():
            if word in col1_counts:
                col1_counts[word] -= count  # Reduce count but not below zero
                if col1_counts[word] <= 0:
                    del col1_counts[word]  # Remove word if count reaches zero

        # Reconstruct the filtered list based on updated counts
        filtered_list = []
        for word, count in col1_counts.items():
            filtered_list.extend([word] * count)  # Add back only remaining occurrences

        return filtered_list

    # remove named entities from matched occupations
    df['overlapping_occupations_sentence1_male'] = df.apply(filter_NER, axis=1)
    # df['overlapping_occupations_sentence1_female'] = df.apply(filter_NER, axis=1)
    # print(df['overlapping_occupations_sentence1_female'].head(10))
    

In [18]:
new_german_analysis(de_female_df)
overlapping_occ_counts_s1_female_all = Counter(chain.from_iterable(de_female_df['overlapping_occupations_sentence1_female']))
overlapping_occ_counts_s1_female_all_male = Counter(chain.from_iterable(de_female_df['overlapping_occupations_sentence1_male']))
print(overlapping_occ_counts_s1_female_all)
print(overlapping_occ_counts_s1_female_all_male)
occ_titles_found_female = de_female_df[de_female_df['overlapping_occupations_sentence1_female'].map(len)>0]
print(occ_titles_found_female.shape[0])

hello
0    [emmi, agathe, karola, wiltraut, rupp-von, brü...
1    [sylvia, bayr-klimpfinger, wien, österreich-un...
2                 [ulrike, scheel, dortmund, deutsche]
3                             [renee, goddard, berlin]
4      [isabella, kossina, wien, österreichische, spö]
5    [florina, gillnik, königswinter, florentine, e...
6    [christina, kauschke, dortmund, philipps-unive...
7    [susana, kasakoff, buenos, aires, argentinisch...
8       [heike, marianne, goetze, osnabrück, deutsche]
9    [irmgard, marie, elisabeth, von, witzleben, ar...
Name: named_entities, dtype: object
0                                   [richterin]
1                    [psychologin, professorin]
2    [bewegungslehrerin, fernsehschauspielerin]
3                              [schauspielerin]
4                                 [politikerin]
5                                            []
6                      [professorin, klinische]
7                                   [pianistin]
8                       

In [27]:
new_german_analysis(de_female_df)
overlapping_occ_counts_s1_female_all = Counter(chain.from_iterable(de_female_df['overlapping_occupations_sentence1_female']))
overlapping_occ_counts_s1_female_all_male = Counter(chain.from_iterable(de_female_df['overlapping_occupations_sentence1_male']))
print(overlapping_occ_counts_s1_female_all)
print(overlapping_occ_counts_s1_female_all_male)
occ_titles_found_female = de_female_df[de_female_df['overlapping_occupations_sentence1_female'].map(len)>0]
print(occ_titles_found_female.shape[0])

hello
0    [emmi, agathe, karola, wiltraut, rupp-von, brü...
1    [sylvia, bayr-klimpfinger, wien, österreich-un...
2                 [ulrike, scheel, dortmund, deutsche]
3                             [renee, goddard, berlin]
4      [isabella, kossina, wien, österreichische, spö]
5    [florina, gillnik, königswinter, florentine, e...
6    [christina, kauschke, dortmund, philipps-unive...
7    [susana, kasakoff, buenos, aires, argentinisch...
8       [heike, marianne, goetze, osnabrück, deutsche]
9    [irmgard, marie, elisabeth, von, witzleben, ar...
Name: named_entities, dtype: object
Counter({'schauspielerin': 4627, 'politikerin': 4392, 'schriftstellerin': 2497, 'autorin': 2037, 'journalistin': 1391, 'sängerin': 1129, 'badmintonspielerin': 913, 'künstlerin': 810, 'fussballspielerin': 783, 'opernsängerin': 640, 'professorin': 611, 'synchronsprecherin': 598, 'malerin': 568, 'historikerin': 546, 'regisseurin': 529, 'übersetzerin': 512, 'theaterschauspielerin': 480, 'hochschullehrerin': 4